In [1]:
import pandas as pd
import numpy as np
from ast import literal_eval

In [2]:
data_prefix = './unprocessed_data'

In [3]:
df_metadata = pd.read_csv(f'{data_prefix}/movies_metadata.csv')
df_metadata = df_metadata.drop([19730, 29503, 35587])

/tmp/ipykernel_16781/2015830793.py:1: DtypeWarning: Columns (10) have mixed types. Specify dtype option on import or set low_memory=False.
  df_metadata = pd.read_csv(f'{data_prefix}/movies_metadata.csv')


Узнаем, какие данные есть в датасете

In [4]:
df_metadata = df_metadata.convert_dtypes()
df_metadata.dtypes

adult                    string[python]
belongs_to_collection    string[python]
budget                   string[python]
genres                   string[python]
homepage                 string[python]
id                       string[python]
imdb_id                  string[python]
original_language        string[python]
original_title           string[python]
overview                 string[python]
popularity                       object
poster_path              string[python]
production_companies     string[python]
production_countries     string[python]
release_date             string[python]
revenue                           Int64
runtime                           Int64
spoken_languages         string[python]
status                   string[python]
tagline                  string[python]
title                    string[python]
video                           boolean
vote_average                    Float64
vote_count                        Int64
dtype: object

In [5]:
df_metadata['id'] = df_metadata['id'].astype(int)

In [6]:
df_metadata.isnull().any()

adult                    False
belongs_to_collection     True
budget                   False
genres                   False
homepage                  True
id                       False
imdb_id                   True
original_language         True
original_title           False
overview                  True
popularity                True
poster_path               True
production_companies      True
production_countries      True
release_date              True
revenue                   True
runtime                   True
spoken_languages          True
status                    True
tagline                   True
title                     True
video                     True
vote_average              True
vote_count                True
dtype: bool

Удаляем следующие столбцы:
* `adults` - поскольку в дальнейшем будет добавлен столбец в возвратным рейтингом фильма MPAA
* `belongs_to_collection`
* `homepage`
* `popularity` - эта метрика [высчитывается](https://developer.themoviedb.org/docs/popularity-and-trending) для фильма каждый день, поэтому она актуальна в тот момент, когда были собраны данные в датасете
* `poster_path`
* `video`

In [7]:
df_metadata = df_metadata.drop(columns=['adult', 'belongs_to_collection', 'homepage', 'popularity', 'poster_path', 'video'], axis=1)

Посмотрим, у скольки записей есть целевая переменная: сборы (`revenue`)

In [8]:
df_metadata[(df_metadata['revenue'].notna()) & (df_metadata['revenue'] != 0)].shape

(7408, 18)

Добавляем в исходный датасет данные, полученные с сайта Box Office Mojo.

In [9]:
df_mojo = pd.read_csv('./scrapper/result.csv')
df_mojo.head()

,imdb_id,domestic_distributor,domestic_opening,budget,mpaa,running_time,genres,domestic_revenue,international_revenue,worldwide_revenue
0,tt0113041,Walt Disney Studios Motion Pictures,11134978,0,PG,1 hr 46 min,Comedy Family Romance,76594107,0,76594107
1,tt0113228,Warner Bros.,7797185,0,PG-13,1 hr 41 min,Comedy Romance,71518503,0,71518503
2,tt0112760,Metro-Goldwyn-Mayer (MGM),2371415,98000000,PG-13,2 hr 4 min,Action Adventure Comedy,10017322,0,10017322
3,tt0114576,Universal Pictures,4782445,0,R,1 hr 51 min,Action Crime Thriller,20350171,44000000,64350171
4,tt0112453,Universal Pictures,1519755,0,NaN,1 hr 18 min,Adventure Animation Drama Family History,11348324,766,11349090


Удаляем поля, которые уже есть в исходном датасете.
`running_time` можно удалять, т.к. в исходных данных он проставлен практическиу всех фильмов.

In [10]:
df_mojo = df_mojo.drop(['domestic_revenue', 'international_revenue', 'running_time', 'genres'], axis=1)
df_mojo.rename(columns={'budget': 'scrapped_budget'}, inplace=True)

In [11]:
df = df_metadata.merge(df_mojo, on=['imdb_id'], how='inner')

Дополняем инфоормацию о бюджете и сборах исходного датасета с помощью данных с Box Office Mojo

In [12]:
def fill_zero_fileds(field_1, field_2):
    def f(row):
        if row[field_1] != 0:
            return row[field_1]
        return row[field_2]
    
    return f

df['new_budget'] = df.apply(fill_zero_fileds('budget', 'scrapped_budget'), axis=1)
df['new_revenue'] = df.apply(fill_zero_fileds('revenue', 'worldwide_revenue'), axis=1)
df = df.drop(columns=['budget', 'scrapped_budget', 'revenue', 'worldwide_revenue'], axis=1)
df = df.rename(columns={'new_budget': 'budget', 'new_revenue': 'revenue'})

In [13]:
df[df['revenue'] != 0].shape

(20260, 21)

Как видно, количество "валидных" записей увеличилось с 7 тыс. до 20 тыс.

Удалим строки, в которых неизвестна целевая переменная: сборы (`revenue`)

In [14]:
df = df[(df['revenue'].notna()) & (df['revenue'] != 0)]

Поскольку столцбы `genres`, `production_companies`, `production_countries` и `spoken_languages` представляют собой список словарей, нужно их обработать отдельно.

In [15]:
def get_value_from_dict(column, key):
    def f(row):
        dicts = literal_eval(row[column])
        res = []
        for c in dicts:
            res.append(c[key])
        if len(res) == 0:
            return np.NAN
        return res

    return f

df['genres'] = df.apply(get_value_from_dict('genres', 'name'), axis=1)
df['production_companies'] = df.apply(get_value_from_dict('production_companies', 'name'), axis=1)
df['production_countries'] = df.apply(get_value_from_dict('production_countries', 'iso_3166_1'), axis=1)
df['spoken_languages'] = df.apply(get_value_from_dict('spoken_languages', 'iso_639_1'), axis=1)

Теперь нужно добавить информацию об актерах и режиссере

In [ ]:
df_credits = pd.read_csv(f'{data_prefix}/credits.csv')

def get_3_actors(row):
    cast = literal_eval(row['cast'])
    cast = sorted(cast, key=lambda x: x['order'])
    cast = list(map(lambda x: x['name'], cast))
    return cast[:3]

def get_director(row):
    crew = literal_eval(row['crew'])
    for c in crew:
        if c['job'] == 'Director':
            return c['name']
    return np.NAN

df_credits['actors'] = df_credits.apply(get_3_actors, axis=1)

df_credits['director'] = df_credits.apply(get_director, axis=1)

df_credits = df_credits.drop(columns=['cast', 'crew'], axis=1)

df = df.merge(df_credits, on=['id'], how='inner')

In [ ]:
df_posters = pd.read_csv(f'{data_prefix}/posters.csv', encoding='latin1')
df_posters = df_posters.rename(columns={"imdbId": "imdb_id", "Poster": "poster"})
df_posters.head()

,imdb_id,Imdb Link,Title,IMDB Score,Genre,poster
0,114709,http://www.imdb.com/title/tt114709,Toy Story (1995),8.3,Animation|Adventure|Comedy,https://images-na.ssl-images-amazon.com/images...
1,113497,http://www.imdb.com/title/tt113497,Jumanji (1995),6.9,Action|Adventure|Family,https://images-na.ssl-images-amazon.com/images...
2,113228,http://www.imdb.com/title/tt113228,Grumpier Old Men (1995),6.6,Comedy|Romance,https://images-na.ssl-images-amazon.com/images...
3,114885,http://www.imdb.com/title/tt114885,Waiting to Exhale (1995),5.7,Comedy|Drama|Romance,https://images-na.ssl-images-amazon.com/images...
4,113041,http://www.imdb.com/title/tt113041,Father of the Bride Part II (1995),5.9,Comedy|Family|Romance,https://images-na.ssl-images-amazon.com/images...


In [ ]:
df_posters['imdb_id'] = df_posters['imdb_id'].astype(str)
df_posters['imdb_id'] = df_posters['imdb_id'].apply(lambda x: 'tt' + x.rjust(7, '0'))
df_posters.head()

In [ ]:
df = df.merge(df_posters[['imdb_id', 'poster']], on=['imdb_id'], how='inner')
df.head()

,genres,id,imdb_id,original_language,original_title,overview,production_companies,production_countries,release_date,runtime,...,vote_average,vote_count,domestic_distributor,domestic_opening,mpaa,budget,revenue,actors,director,poster
0,"[Animation, Comedy, Family]",862,tt0114709,en,Toy Story,"Led by Woody, Andy's toys live happily in his ...",[Pixar Animation Studios],[US],1995-10-30,81,...,7.7,5415,Walt Disney Studios Motion Pictures,29140617,NaN,30000000,373554033,"[Tom Hanks, Tim Allen, Don Rickles]",John Lasseter,https://images-na.ssl-images-amazon.com/images...
1,"[Adventure, Fantasy, Family]",8844,tt0113497,en,Jumanji,When siblings Judy and Peter discover an encha...,"[TriStar Pictures, Teitler Film, Interscope Co...",[US],1995-12-15,104,...,6.9,2413,Sony Pictures Releasing,11084370,PG,65000000,262797249,"[Robin Williams, Jonathan Hyde, Kirsten Dunst]",Joe Johnston,https://images-na.ssl-images-amazon.com/images...
2,"[Romance, Comedy]",15602,tt0113228,en,Grumpier Old Men,A family wedding reignites the ancient feud be...,"[Warner Bros., Lancaster Gate]",[US],1995-12-22,101,...,6.5,92,Warner Bros.,7797185,PG-13,0,71518503,"[Walter Matthau, Jack Lemmon, Ann-Margret]",Howard Deutch,https://images-na.ssl-images-amazon.com/images...
3,"[Comedy, Drama, Romance]",31357,tt0114885,en,Waiting to Exhale,"Cheated on, mistreated and stepped on, the wom...",[Twentieth Century Fox Film Corporation],[US],1995-12-22,127,...,6.1,34,Twentieth Century Fox,14126927,R,16000000,81452156,"[Whitney Houston, Angela Bassett, Loretta Devine]",Forest Whitaker,https://images-na.ssl-images-amazon.com/images...
4,[Comedy],11862,tt0113041,en,Father of the Bride Part II,Just when George Banks has recovered from his ...,"[Sandollar Productions, Touchstone Pictures]",[US],1995-02-10,106,...,5.7,173,Walt Disney Studios Motion Pictures,11134978,PG,0,76578911,"[Steve Martin, Diane Keaton, Martin Short]",Charles Shyer,https://images-na.ssl-images-amazon.com/images...


In [ ]:
df[(df['poster'].notna())].shape

(18833, 24)

In [ ]:
df.to_csv('./movie_dataset.csv')